## YOLO format

: 어디에 어떤 클래스가 있는지, ROI를 지정하는 게 레이블링이다.  
: class id, center x, center y, width, height 형식

- B-Box == Boundary box  

### cfg 수정할 부분
: max_batches == class_num * 2000  (**epochs 횟수** 정하는 **대신 max_batches 횟수**만큼 돌게함)  
: steps == max_batches * 0.8, max_batche * 0.9  
:  [Yolo]위에 있는 [Convolusion] 에서 filters 개수만 바꾸면 됨  
  
**(클래스 개수 + 좌표 개수 + 1) * 채널 == (2+4+1)*3**

### cfg 설명

## YOLO 데이터 파일 만들기

1. classes.names 만들기  
: DNN에서 사용되는 클래스 이름을 넣어야 한다  

```
classes.names

apple
tangerine
```

2. my_yolo.data  
: yolo 실행시 참조하는 메타데이터 파일

```
classes : 클래스 개수
train : 훈련 데이터 리스트 위치
valid : 검증 데이터 리스트 위치
names : classes.names 파일 위치
backup : 훈련 가중치 백업 위치
```

In [20]:
# 경로 설정
!ls -al ./darknet

darknet_path = '/home/jaehoon/jupyter_file/cv_lecture/darknet'
data_path = darknet_path + '/my_yolo/data'

yolo_image_path = data_path
yolo_format_path = data_path

ls: './darknet'에 접근할 수 없습니다: 그런 파일이나 디렉터리가 없습니다


In [21]:
# cfg 설정
class_count = 0  # 클래스 개수 (추후에 업데이트 할 것)
test_ratio = 0.2 # 테스트셋 비율
paths = []       # 파일 경로

#### classes.names 생성

In [22]:
# classes.names 생성 & class_count 업데이트
with open(yolo_format_path + '/classes.names', 'w') as names, open(yolo_format_path + '/classes.txt') as txt:
    for line in txt:
        names.write(line)
        class_count += 1
        
    print('[classes.names] is created')

[classes.names] is created


#### meta data 생성

In [23]:
with open(yolo_format_path + '/custom_data.data', 'w') as data:
    data.write('classes = ' + str(class_count) + '\n')
    data.write('train = ' + yolo_format_path + '/train.txt' + '\n') # 훈련 데이터 위치 (나중에 추가할 것)
    data.write('valid = ' + yolo_format_path + '/test.txt' + '\n')
    data.write('names = ' + yolo_format_path + '/classes.names' + '\n')
    data.write('backup = backup') # 다크넷 실행 파일 위치에 backup폴더가 있어야 한다
    
    print('[custom_data.data] is created')

[custom_data.data] is created


#### train / test 데이터 지정하기

In [24]:
# paths 저장

import os
for current_dir, dirs, files in os.walk(yolo_image_path):
    # print(current_dir, dirs, files)
    for file in files:
        if file.endswith('.jpg'):
            image_path = yolo_image_path + '/' + file
            print(image_path)
            
            paths.append(image_path + '\n')

/home/jaehoon/jupyter_file/cv_lecture/darknet/my_yolo/data/fruit07.jpg
/home/jaehoon/jupyter_file/cv_lecture/darknet/my_yolo/data/fruit04.jpg
/home/jaehoon/jupyter_file/cv_lecture/darknet/my_yolo/data/fruit03.jpg
/home/jaehoon/jupyter_file/cv_lecture/darknet/my_yolo/data/fruit01.jpg
/home/jaehoon/jupyter_file/cv_lecture/darknet/my_yolo/data/fruit10.jpg
/home/jaehoon/jupyter_file/cv_lecture/darknet/my_yolo/data/fruit02.jpg
/home/jaehoon/jupyter_file/cv_lecture/darknet/my_yolo/data/fruit08.jpg
/home/jaehoon/jupyter_file/cv_lecture/darknet/my_yolo/data/fruit09.jpg
/home/jaehoon/jupyter_file/cv_lecture/darknet/my_yolo/data/fruit06.jpg
/home/jaehoon/jupyter_file/cv_lecture/darknet/my_yolo/data/fruit05.jpg


In [25]:
# train, test 분리
num_test = int(len(paths) * test_ratio)
paths_test = paths[:num_test]
paths_train = paths[num_test:]

# train.txt
with open(yolo_format_path + '/train.txt', 'w') as train_txt:
    for path in paths_train:
        train_txt.write(path)
    print('[train.txt] is created')
    
# test.txt
with open(yolo_format_path + '/test.txt', 'w') as test_txt:
    for path in paths_test:
        test_txt.write(path)
    print('[test.txt] is created')

[train.txt] is created
[test.txt] is created


## Default Darknet YOLO 

In [26]:
# !wget https://pjreddie.com/media/files/yolov3.weights

In [27]:
# 다크넷 실행파일 확인
!ls -al '/home/jaehoon/jupyter_file/cv_lecture/darknet/bin'

# 실행 위치 변경
os.chdir('/home/jaehoon/jupyter_file/cv_lecture/darknet')

# 현재 디렉토리
print(os.path.abspath(os.curdir))

# 다크넷 파일 실행 권한 변경
!chmod +x ./bin/darknet

합계 728
drwxrwxrwx  3 jaehoon jaehoon   4096  7월 23 09:02 .
drwxrwxrwx 12 jaehoon jaehoon   4096  7월 22 17:58 ..
drwxrwxr-x  2 jaehoon jaehoon   4096  7월 23 09:02 backup
-rwxrwxrwx  1 jaehoon jaehoon 732648  7월 22 17:01 darknet
/home/jaehoon/jupyter_file/cv_lecture/darknet


In [28]:
!./bin/darknet detect cfg/yolov3.cfg weights/yolov3.weights my_yolo/data/fruit10.jpg

layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32  0.299 BFLOPs
    1 conv     64  3 x 3 / 2   416 x 416 x  32   ->   208 x 208 x  64  1.595 BFLOPs
    2 conv     32  1 x 1 / 1   208 x 208 x  64   ->   208 x 208 x  32  0.177 BFLOPs
    3 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64  1.595 BFLOPs
    4 res    1                 208 x 208 x  64   ->   208 x 208 x  64
    5 conv    128  3 x 3 / 2   208 x 208 x  64   ->   104 x 104 x 128  1.595 BFLOPs
    6 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64  0.177 BFLOPs
    7 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128  1.595 BFLOPs
    8 res    5                 104 x 104 x 128   ->   104 x 104 x 128
    9 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64  0.177 BFLOPs
   10 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128  1.595 BFLOPs
   11 res    8                 104 x 

Loading weights from weights/yolov3.weights...Done!
my_yolo/data/fruit10.jpg: Predicted in 4.497950 seconds.
orange: 100%
orange: 99%
orange: 98%


In [29]:
# 띠용! 훈련 안 했더니 인식을 못하네
import cv2

img = cv2.imread('predictions.jpg')
img = cv2.resize(img, (600, 416))
cv2.imshow('pred', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [31]:
# 백업 폴더 만들기
!rm -rf /home/jaehoon/jupyter_file/cv_lecture/darknet/bin/backup
!mkdir ./bin/backup # 훈련용 backup폴더는 bin 안에 따로 만들어야 한다

In [32]:
# 훈련 : 백업이 자동으로 생김
# detector 와 train은 darknet.exe 안에 들어있는 arguments
!./bin/darknet detector train my_yolo/data/custom_data.data my_yolo/cfg/custom-train-yolo.cfg weights/darknet53.conv.74 -dont_show

custom-train-yolo
layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32  0.299 BFLOPs
    1 conv     64  3 x 3 / 2   416 x 416 x  32   ->   208 x 208 x  64  1.595 BFLOPs
    2 conv     32  1 x 1 / 1   208 x 208 x  64   ->   208 x 208 x  32  0.177 BFLOPs
    3 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64  1.595 BFLOPs
    4 res    1                 208 x 208 x  64   ->   208 x 208 x  64
    5 conv    128  3 x 3 / 2   208 x 208 x  64   ->   104 x 104 x 128  1.595 BFLOPs
    6 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64  0.177 BFLOPs
    7 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128  1.595 BFLOPs
    8 res    5                 104 x 104 x 128   ->   104 x 104 x 128
    9 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64  0.177 BFLOPs
   10 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128  1.595 BFLOPs
   11 res    8     

Loading weights from weights/darknet53.conv.74...Done!
Learning Rate: 0.001, Momentum: 0.9, Decay: 0.0005
Resizing
384
Killed


In [37]:
# mean average precision 계산 : .data & cfg & backup weights
!./bin/darknet detector map my_yolo/data/custom_data.data my_yolo/cfg/custom-train-yolo.cfg backup/custom-train-yolo_final.weights -dont_show

In [34]:
!./bin/darknet detector test my_yolo/data/custom_data.data my_yolo/cfg/custom-train-yolo.cfg backup/custom-train-yolo_final.weights my_yolo/data/fruit10.jpg -dont_show

layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32  0.299 BFLOPs
    1 conv     64  3 x 3 / 2   416 x 416 x  32   ->   208 x 208 x  64  1.595 BFLOPs
    2 conv     32  1 x 1 / 1   208 x 208 x  64   ->   208 x 208 x  32  0.177 BFLOPs
    3 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64  1.595 BFLOPs
    4 res    1                 208 x 208 x  64   ->   208 x 208 x  64
    5 conv    128  3 x 3 / 2   208 x 208 x  64   ->   104 x 104 x 128  1.595 BFLOPs
    6 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64  0.177 BFLOPs
    7 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128  1.595 BFLOPs
    8 res    5                 104 x 104 x 128   ->   104 x 104 x 128
    9 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64  0.177 BFLOPs
   10 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128  1.595 BFLOPs
   11 res    8                 104 x 

Loading weights from backup/custom-train-yolo_final.weights...Done!
my_yolo/data/fruit10.jpg: Predicted in 4.689630 seconds.
tangerine: 100%
tangerine: 100%
apple: 100%


In [35]:
# 결과
img = cv2.imread('predictions.jpg')
img = cv2.resize(img, (600, 416))
cv2.imshow('pred', img)
cv2.waitKey(0)
cv2.destroyAllWindows()